In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px # to plot the time series plot
from sklearn import metrics # for the evaluation
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import tensorflow as tf 
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
import os
import datetime
import IPython
import IPython.display

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import pandas as pd
from pandas import read_csv
import numpy as np
import random
from pandas import DataFrame
from numpy import concatenate
from pandas import concat
from scipy.stats import randint as sp_randint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score 


import matplotlib.patches as mpatches
import matplotlib.pylab as plt
import matplotlib as mpl
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 7)
plt.style.use('fast')
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False
from sklearn.neural_network import MLPRegressor


np.random.seed(123) # for reproducibility
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Flatten
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from scikeras.wrappers import KerasClassifier, KerasRegressor
import keras_tuner

In [2]:
# CARGAR LA BASE DE DATOS
from pandas import read_csv
datos = read_csv('Piedecuesta_Lags.csv', encoding='latin-1', sep=";")
# Dropping the id and date columns
datos.columns

Index(['Municipio', 'Año', 'Semana_Epi', 'Casos_Dengue', 'Prom_Dia_Preci',
       'Prom_Dia_Preci_1', 'Prom_Dia_Preci_2', 'Prom_Dia_Preci_3',
       'Prom_Dia_Preci_4', 'Prom_Dia_Preci_5', 'Prom_Dia_Preci_6',
       'Total_Preci_Sem', 'Total_Preci_Sem_1', 'Total_Preci_Sem_2',
       'Total_Preci_Sem_3', 'Total_Preci_Sem_4', 'Total_Preci_Sem_5',
       'Total_Preci_Sem_6'],
      dtype='object')

In [3]:
datos = datos.drop(['Municipio', 'Año', 'Semana_Epi','Prom_Dia_Preci_1', 'Prom_Dia_Preci_2', 'Prom_Dia_Preci_3',
       'Prom_Dia_Preci_4', 'Prom_Dia_Preci_5', 'Prom_Dia_Preci_6', 'Total_Preci_Sem_1', 'Total_Preci_Sem_2',
       'Total_Preci_Sem_3', 'Total_Preci_Sem_4', 'Total_Preci_Sem_5','Total_Preci_Sem_6'],axis=1)
datos.head(3)

,Casos_Dengue,Prom_Dia_Preci,Total_Preci_Sem
0,8,0.00,0.0
1,3,6.24,43.7
2,2,7.11,49.8


In [4]:
train_size = int(len(datos) * 0.7)
test_size = len(datos) - train_size
train, test = datos.iloc[0:train_size], datos.iloc[train_size:len(datos)]
print(len(train), len(test))

473 203


In [5]:
f_columns = ['Prom_Dia_Preci', 'Total_Preci_Sem']
f_transformer = MinMaxScaler(feature_range=(0, 1))
f_transformer = f_transformer.fit(train[f_columns].to_numpy())
train.loc[:, f_columns] = f_transformer.transform(train[f_columns].to_numpy())
test.loc[:, f_columns] = f_transformer.transform(test[f_columns].to_numpy())

In [6]:
dengue_transformer = MinMaxScaler(feature_range=(0, 1))
dengue_transformer = dengue_transformer.fit(train[['Casos_Dengue']])
train['Casos_Dengue'] = dengue_transformer.transform(train[['Casos_Dengue']])
test['Casos_Dengue'] = dengue_transformer.transform(test[['Casos_Dengue']])

In [7]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [8]:
time_steps = 10
# reshape to [samples, time_steps, n_features]
X_train, y_train = create_dataset(train, train.Casos_Dengue, time_steps)
X_test, y_test = create_dataset(test, test.Casos_Dengue, time_steps)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape )

(463, 10, 3) (463,) (193, 10, 3) (193,)


In [9]:
from tensorflow.keras.layers import InputLayer

def LSTM_model(neuronas,
               dropout_rate,
               num_layers,
               activation = 'relu',
               init_mode='uniform',
               #optimizer='adam'
              ):
    
    model = keras.Sequential()
    
    # CAPA DE ENTRADA
    model.add(keras.layers.InputLayer(input_shape=(X_train.shape[1], X_train.shape[2])))
    
    
    #CAPAS INTERMEDIAS
    for i in range(num_layers):
        model.add(keras.layers.LSTM(units=neuronas,
                                    activation=activation,
                                    return_sequences = True,
                                    dropout=dropout_rate,
                                    kernel_initializer=init_mode))
        
        
    #CAPA DE SALIDA
    model.add(keras.layers.Dense(1, activation="linear"))
    
    #COMPILACIÓN DEL MODELO
    model.compile(optimizer="adam", loss = 'mse', metrics = ['mse'])
    
    return model

In [10]:
#HIPERPARÁMETROS

neuronas = [50, 70, 100]
activation = ['sigmoid','relu', 'tanh']
init_mode = ['uniform', 'normal', 'zero']
num_layers = [3, 5, 7]
dropout_rate = [0.1, 0.3, 0.5]

#batch_size = [40, 60, 80]
#epochs = [10, 50, 100]
#learning_rate = [0.1, 0.2, 0.3],
#momentum = [0.2, 0.4, 0.5]

param_grid = dict(model__neuronas=neuronas,
                  model__activation=activation,
                  model__init_mode=init_mode,
                  model__dropout_rate=dropout_rate,
                  model__num_layers=num_layers)
                  #batch_size=batch_size,
                  #epochs=epochs,
                  #optimizer__learning_rate=learning_rate,
                  #optimizer__momentum=momentum

In [11]:
model =  KerasRegressor(model=LSTM_model, verbose=0)

rs = RandomizedSearchCV(estimator=model, param_distributions=param_grid, scoring='neg_mean_squared_error',
                        n_jobs=-1, cv=5, n_iter=20, verbose=0,random_state = 123, return_train_score = True)

In [12]:
fitted_model = rs.fit(X_train, y_train)

In [13]:
resultados = pd.DataFrame(fitted_model.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)')\
    .drop(columns = 'params')\
    .sort_values('mean_test_score', ascending = False)\
    .head(10)

,param_model__num_layers,param_model__neuronas,param_model__init_mode,param_model__dropout_rate,param_model__activation,mean_test_score,std_test_score,mean_train_score,std_train_score
0,3,100,uniform,0.5,tanh,NaN,NaN,NaN,NaN
1,5,50,zero,0.1,sigmoid,NaN,NaN,NaN,NaN
2,3,100,normal,0.5,relu,NaN,NaN,NaN,NaN
3,5,50,uniform,0.3,tanh,NaN,NaN,NaN,NaN
4,5,70,uniform,0.3,sigmoid,NaN,NaN,NaN,NaN
5,5,50,uniform,0.1,tanh,NaN,NaN,NaN,NaN
6,7,70,uniform,0.3,tanh,NaN,NaN,NaN,NaN
7,7,100,zero,0.5,sigmoid,NaN,NaN,NaN,NaN
8,5,50,normal,0.1,relu,NaN,NaN,NaN,NaN
9,3,100,zero,0.1,sigmoid,NaN,NaN,NaN,NaN


In [14]:
print("Best: %f using %s" % (fitted_model.best_score_, fitted_model.best_params_))

Best: nan using {'model__num_layers': 3, 'model__neuronas': 100, 'model__init_mode': 'uniform', 'model__dropout_rate': 0.5, 'model__activation': 'tanh'}
